# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.model_selection import train_test_split

In [2]:
pr = pd.read_csv('previsao_de_renda.csv')

In [3]:
pr

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,14996,2016-03-01,3722,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [4]:
pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [5]:
pdr = pr.drop(['Unnamed: 0'], axis=1)
pdr

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,2016-03-01,3722,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [6]:
X =  pdr
y =  pdr

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.25, train_size=.75)

In [30]:
modelo = 'id_cliente ~ C(tipo_renda) +  np.log(renda) + sexo + idade'
md = smf.ols(modelo, data = pr)
reg = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 0.01
                        , alpha =  0)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             id_cliente   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     10.26
Date:                Fri, 06 Jan 2023   Prob (F-statistic):           2.04e-14
Time:                        11:10:39   Log-Likelihood:            -1.4838e+05
No. Observations:               15000   AIC:                         2.968e+05
Df Residuals:                   14992   BIC:                         2.968e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1.012e+04    406.440     24.887      0.000    9318.464    1.09e+04
C(tipo_renda)[T.Bolsista]         -7539.4143   1597.147     -4.721      0.000   -1.07e+04   -4408.811
C(tipo_renda)[T.Empresário]         -57.1132     97.963     -0.583      0.560    -249.134     134.907
C(tipo_renda)[T.Pensionista]       -207.4333    138.201     -1.501      0.133    -478.325      63.458
C(tipo_renda)[T.Servidor público]     5.2709    145.816      0.036      0.971    -280.547     291.089
sexo[T.M]                           191.5159     93.234      2.054      0.040       8.766     374.265
np.log(renda)                      -122.0249     48.876     -2.497      0.013    -217.829     -26.221
idade                               -18.6369      4.569     -4.079      0.000     -27.593      -9.681
==============================================================================
Omnibus:                    10392.421   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.783
Skew:                           0.002   Prob(JB):                    2.00e-187
Kurtosis:                       1.827   Cond. No.                     1.88e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [32]:
modelo = 'id_cliente ~ C(tipo_renda) +  np.log(renda) + sexo + idade'
md = smf.ols(modelo, data = pr)
reg1 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 0.01
                        , alpha =  0.001)
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             id_cliente   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     10.26
Date:                Fri, 06 Jan 2023   Prob (F-statistic):           2.04e-14
Time:                        11:14:26   Log-Likelihood:            -1.4838e+05
No. Observations:               15000   AIC:                         2.968e+05
Df Residuals:                   14992   BIC:                         2.968e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1.012e+04    406.440     24.887      0.000    9318.464    1.09e+04
C(tipo_renda)[T.Bolsista]         -7539.4143   1597.147     -4.721      0.000   -1.07e+04   -4408.811
C(tipo_renda)[T.Empresário]         -57.1132     97.963     -0.583      0.560    -249.134     134.907
C(tipo_renda)[T.Pensionista]       -207.4333    138.201     -1.501      0.133    -478.325      63.458
C(tipo_renda)[T.Servidor público]     5.2709    145.816      0.036      0.971    -280.547     291.089
sexo[T.M]                           191.5159     93.234      2.054      0.040       8.766     374.265
np.log(renda)                      -122.0249     48.876     -2.497      0.013    -217.829     -26.221
idade                               -18.6369      4.569     -4.079      0.000     -27.593      -9.681
==============================================================================
Omnibus:                    10392.421   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.783
Skew:                           0.002   Prob(JB):                    2.00e-187
Kurtosis:                       1.827   Cond. No.                     1.88e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [33]:
modelo = 'id_cliente ~ C(tipo_renda) +  np.log(renda) + sexo + idade'
md = smf.ols(modelo, data = pr)
reg2 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 0.01
                        , alpha =  0.005)
reg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             id_cliente   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     10.26
Date:                Fri, 06 Jan 2023   Prob (F-statistic):           2.04e-14
Time:                        11:14:49   Log-Likelihood:            -1.4838e+05
No. Observations:               15000   AIC:                         2.968e+05
Df Residuals:                   14992   BIC:                         2.968e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1.012e+04    406.440     24.887      0.000    9318.464    1.09e+04
C(tipo_renda)[T.Bolsista]         -7539.4143   1597.147     -4.721      0.000   -1.07e+04   -4408.811
C(tipo_renda)[T.Empresário]         -57.1132     97.963     -0.583      0.560    -249.134     134.907
C(tipo_renda)[T.Pensionista]       -207.4333    138.201     -1.501      0.133    -478.325      63.458
C(tipo_renda)[T.Servidor público]     5.2709    145.816      0.036      0.971    -280.547     291.089
sexo[T.M]                           191.5159     93.234      2.054      0.040       8.766     374.265
np.log(renda)                      -122.0249     48.876     -2.497      0.013    -217.829     -26.221
idade                               -18.6369      4.569     -4.079      0.000     -27.593      -9.681
==============================================================================
Omnibus:                    10392.421   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.783
Skew:                           0.002   Prob(JB):                    2.00e-187
Kurtosis:                       1.827   Cond. No.                     1.88e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [34]:
modelo = 'id_cliente ~ C(tipo_renda) +  np.log(renda) + sexo + idade'
md = smf.ols(modelo, data = pr)
reg3 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 0.01
                        , alpha =  0.01)
reg3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             id_cliente   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     10.26
Date:                Fri, 06 Jan 2023   Prob (F-statistic):           2.04e-14
Time:                        11:15:09   Log-Likelihood:            -1.4838e+05
No. Observations:               15000   AIC:                         2.968e+05
Df Residuals:                   14992   BIC:                         2.968e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1.012e+04    406.440     24.887      0.000    9318.464    1.09e+04
C(tipo_renda)[T.Bolsista]         -7539.4143   1597.147     -4.721      0.000   -1.07e+04   -4408.811
C(tipo_renda)[T.Empresário]         -57.1132     97.963     -0.583      0.560    -249.134     134.907
C(tipo_renda)[T.Pensionista]       -207.4333    138.201     -1.501      0.133    -478.325      63.458
C(tipo_renda)[T.Servidor público]     5.2709    145.816      0.036      0.971    -280.547     291.089
sexo[T.M]                           191.5159     93.234      2.054      0.040       8.766     374.265
np.log(renda)                      -122.0249     48.876     -2.497      0.013    -217.829     -26.221
idade                               -18.6369      4.569     -4.079      0.000     -27.593      -9.681
==============================================================================
Omnibus:                    10392.421   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.783
Skew:                           0.002   Prob(JB):                    2.00e-187
Kurtosis:                       1.827   Cond. No.                     1.88e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
modelo = 'id_cliente ~ C(tipo_renda) +  np.log(renda) + sexo + idade'
md = smf.ols(modelo, data = pr)
reg4 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 0.01
                        , alpha =  0.05)
reg4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             id_cliente   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     10.26
Date:                Fri, 06 Jan 2023   Prob (F-statistic):           2.04e-14
Time:                        11:15:18   Log-Likelihood:            -1.4838e+05
No. Observations:               15000   AIC:                         2.968e+05
Df Residuals:                   14992   BIC:                         2.968e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1.012e+04    406.440     24.887      0.000    9318.464    1.09e+04
C(tipo_renda)[T.Bolsista]         -7539.4143   1597.147     -4.721      0.000   -1.07e+04   -4408.811
C(tipo_renda)[T.Empresário]         -57.1132     97.963     -0.583      0.560    -249.134     134.907
C(tipo_renda)[T.Pensionista]       -207.4333    138.201     -1.501      0.133    -478.325      63.458
C(tipo_renda)[T.Servidor público]     5.2709    145.816      0.036      0.971    -280.547     291.089
sexo[T.M]                           191.5159     93.234      2.054      0.040       8.766     374.265
np.log(renda)                      -122.0249     48.876     -2.497      0.013    -217.829     -26.221
idade                               -18.6369      4.569     -4.079      0.000     -27.593      -9.681
==============================================================================
Omnibus:                    10392.421   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.783
Skew:                           0.002   Prob(JB):                    2.00e-187
Kurtosis:                       1.827   Cond. No.                     1.88e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [47]:
modelo = 'id_cliente ~ C(tipo_renda) +  np.log(renda) + sexo + idade'
md = smf.ols(modelo, data = pr)
reg5 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 0.01
                        , alpha =  0.1)
reg5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             id_cliente   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     10.26
Date:                Fri, 06 Jan 2023   Prob (F-statistic):           2.04e-14
Time:                        11:21:02   Log-Likelihood:            -1.4838e+05
No. Observations:               15000   AIC:                         2.968e+05
Df Residuals:                   14992   BIC:                         2.968e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1.012e+04    406.440     24.887      0.000    9318.464    1.09e+04
C(tipo_renda)[T.Bolsista]         -7539.4143   1597.147     -4.721      0.000   -1.07e+04   -4408.811
C(tipo_renda)[T.Empresário]         -57.1132     97.963     -0.583      0.560    -249.134     134.907
C(tipo_renda)[T.Pensionista]       -207.4333    138.201     -1.501      0.133    -478.325      63.458
C(tipo_renda)[T.Servidor público]     5.2709    145.816      0.036      0.971    -280.547     291.089
sexo[T.M]                           191.5159     93.234      2.054      0.040       8.766     374.265
np.log(renda)                      -122.0249     48.876     -2.497      0.013    -217.829     -26.221
idade                               -18.6369      4.569     -4.079      0.000     -27.593      -9.681
==============================================================================
Omnibus:                    10392.421   Durbin-Watson:                   2.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.783
Skew:                           0.002   Prob(JB):                    2.00e-187
Kurtosis:                       1.827   Cond. No.                     1.88e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [39]:
modelo = 'np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + qt_pessoas_residencia + qtd_filhos + tempo_emprego'
md = smf.ols(modelo, data = pr)
reg6 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 1
                        , alpha =  0)
reg6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     284.7
Date:                Fri, 06 Jan 2023   Prob (F-statistic):               0.00
Time:                        11:18:40   Log-Likelihood:                -13590.
No. Observations:               12427   AIC:                         2.723e+04
Df Residuals:                   12403   BIC:                         2.741e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               6.7582      0.234     28.874      0.000       6.299       7.217
C(sexo)[T.M]                            0.7815      0.015     53.327      0.000       0.753       0.810
C(posse_de_veiculo)[T.True]             0.0400      0.014      2.828      0.005       0.012       0.068
C(posse_de_imovel)[T.True]              0.0904      0.014      6.470      0.000       0.063       0.118
C(tipo_renda)[T.Bolsista]               0.2562      0.242      1.061      0.289      -0.217       0.730
C(tipo_renda)[T.Empresário]             0.1525      0.015     10.194      0.000       0.123       0.182
C(tipo_renda)[T.Pensionista]           -0.3247      0.242     -1.344      0.179      -0.798       0.149
C(tipo_renda)[T.Servidor público]       0.0563      0.022      2.528      0.011       0.013       0.100
C(educacao)[T.Pós graduação]            0.1306      0.159      0.820      0.412      -0.181       0.443
C(educacao)[T.Secundário]               0.0036      0.072      0.050      0.960      -0.138       0.145
C(educacao)[T.Superior completo]        0.1019      0.072      1.407      0.159      -0.040       0.244
C(educacao)[T.Superior incompleto]     -0.0553      0.078     -0.710      0.478      -0.208       0.097
C(estado_civil)[T.Separado]             0.3425      0.111      3.075      0.002       0.124       0.561
C(estado_civil)[T.Solteiro]             0.2571      0.109      2.356      0.018       0.043       0.471
C(estado_civil)[T.União]               -0.0459      0.025     -1.824      0.068      -0.095       0.003
C(estado_civil)[T.Viúvo]                0.4062      0.116      3.512      0.000       0.179       0.633
C(tipo_residencia)[T.Casa]             -0.0294      0.054     -0.542      0.588      -0.136       0.077
C(tipo_residencia)[T.Com os pais]      -0.0349      0.060     -0.578      0.563      -0.153       0.084
C(tipo_residencia)[T.Comunitário]      -0.0951      0.105     -0.901      0.367      -0.302       0.112
C(tipo_residencia)[T.Estúdio]           0.0921      0.099      0.926      0.354      -0.103       0.287
C(tipo_residencia)[T.Governamental]    -0.0350      0.066     -0.530      0.596      -0.164       0.094
qt_pessoas_residencia                   0.2976      0.108      2.757      0.006       0.086       0.509
qtd_filhos                             -0.2868      0.108     -2.654      0.008      -0.499      -0.075
tempo_emprego                           0.0637      0.001     64.038      0.000       0.062       0.066
==============================================================================
Omnibus:                        0.592   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.744   Jarque-Bera (JB):                0.577
Skew:                           0.016 

In [48]:
modelo = 'np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + qt_pessoas_residencia + qtd_filhos + tempo_emprego'
md = smf.ols(modelo, data = pr)
reg7 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 1
                        , alpha =  0.001)
reg7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     522.9
Date:                Fri, 06 Jan 2023   Prob (F-statistic):               0.00
Time:                        11:24:35   Log-Likelihood:                -13603.
No. Observations:               12427   AIC:                         2.723e+04
Df Residuals:                   12414   BIC:                         2.734e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               7.4467      0.078     95.735      0.000       7.294       7.599
C(sexo)[T.M]                            0.7783      0.015     53.414      0.000       0.750       0.807
C(posse_de_veiculo)[T.True]             0.0393      0.014      2.778      0.005       0.012       0.067
C(posse_de_imovel)[T.True]              0.0919      0.014      6.723      0.000       0.065       0.119
C(tipo_renda)[T.Bolsista]                    0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]             0.1445      0.015      9.900      0.000       0.116       0.173
C(tipo_renda)[T.Pensionista]                 0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]            0          0        nan        nan           0           0
C(educacao)[T.Pós graduação]                 0          0        nan        nan           0           0
C(educacao)[T.Secundário]              -0.0195      0.064     -0.304      0.761      -0.146       0.107
C(educacao)[T.Superior completo]        0.0811      0.065      1.255      0.210      -0.046       0.208
C(educacao)[T.Superior incompleto]     -0.0802      0.071     -1.133      0.257      -0.219       0.059
C(estado_civil)[T.Separado]                  0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]            -0.0865      0.029     -3.026      0.002      -0.142      -0.030
C(estado_civil)[T.União]               -0.0484      0.025     -1.925      0.054      -0.098       0.001
C(estado_civil)[T.Viúvo]                     0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                   0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]          0          0        nan        nan           0           0
qt_pessoas_residencia                  -0.0459      0.023     -1.968      0.049      -0.092      -0.000
qtd_filhos                              0.0555      0.025      2.199      0.028       0.006       0.105
tempo_emprego                           0.0640      0.001     64.997      0.000       0.062       0.066
==============================================================================
Omnibus:                        0.581   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.748   Jarque-Bera (JB):                0.560
Skew:                           0.015 

In [41]:
modelo = 'np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + qt_pessoas_residencia + qtd_filhos + tempo_emprego'
md = smf.ols(modelo, data = pr)
reg6 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 1
                        , alpha =  0.005)
reg6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.348
Method:                 Least Squares   F-statistic:                     829.0
Date:                Fri, 06 Jan 2023   Prob (F-statistic):               0.00
Time:                        11:19:51   Log-Likelihood:                -13658.
No. Observations:               12427   AIC:                         2.733e+04
Df Residuals:                   12419   BIC:                         2.740e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               7.4777      0.023    318.752      0.000       7.432       7.524
C(sexo)[T.M]                            0.7696      0.015     52.838      0.000       0.741       0.798
C(posse_de_veiculo)[T.True]             0.0425      0.014      3.003      0.003       0.015       0.070
C(posse_de_imovel)[T.True]              0.0928      0.014      6.764      0.000       0.066       0.120
C(tipo_renda)[T.Bolsista]                    0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                  0          0        nan        nan           0           0
C(tipo_renda)[T.Pensionista]                 0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]            0          0        nan        nan           0           0
C(educacao)[T.Pós graduação]                 0          0        nan        nan           0           0
C(educacao)[T.Secundário]              -0.1073      0.014     -7.866      0.000      -0.134      -0.081
C(educacao)[T.Superior completo]             0          0        nan        nan           0           0
C(educacao)[T.Superior incompleto]     -0.1631      0.033     -5.011      0.000      -0.227      -0.099
C(estado_civil)[T.Separado]                  0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                  0          0        nan        nan           0           0
C(estado_civil)[T.União]                     0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                     0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                   0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]          0          0        nan        nan           0           0
qt_pessoas_residencia                   0.0048      0.007      0.687      0.492      -0.009       0.019
qtd_filhos                                   0          0        nan        nan           0           0
tempo_emprego                           0.0631      0.001     64.183      0.000       0.061       0.065
==============================================================================
Omnibus:                        0.628   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.731   Jarque-Bera (JB):                0.615
Skew:                           0.017 

In [42]:
modelo = 'np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + qt_pessoas_residencia + qtd_filhos + tempo_emprego'
md = smf.ols(modelo, data = pr)
reg6 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 1
                        , alpha =  0.01)
reg6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1097.
Date:                Fri, 06 Jan 2023   Prob (F-statistic):               0.00
Time:                        11:20:00   Log-Likelihood:                -13675.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12421   BIC:                         2.742e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               7.4681      0.023    325.888      0.000       7.423       7.513
C(sexo)[T.M]                            0.7819      0.014     56.732      0.000       0.755       0.809
C(posse_de_veiculo)[T.True]                  0          0        nan        nan           0           0
C(posse_de_imovel)[T.True]              0.0968      0.014      7.054      0.000       0.070       0.124
C(tipo_renda)[T.Bolsista]                    0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                  0          0        nan        nan           0           0
C(tipo_renda)[T.Pensionista]                 0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]            0          0        nan        nan           0           0
C(educacao)[T.Pós graduação]                 0          0        nan        nan           0           0
C(educacao)[T.Secundário]              -0.0934      0.013     -7.087      0.000      -0.119      -0.068
C(educacao)[T.Superior completo]             0          0        nan        nan           0           0
C(educacao)[T.Superior incompleto]           0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                  0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                  0          0        nan        nan           0           0
C(estado_civil)[T.União]                     0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                     0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                   0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]          0          0        nan        nan           0           0
qt_pessoas_residencia                   0.0069      0.007      0.988      0.323      -0.007       0.021
qtd_filhos                                   0          0        nan        nan           0           0
tempo_emprego                           0.0631      0.001     64.213      0.000       0.061       0.065
==============================================================================
Omnibus:                        0.656   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.720   Jarque-Bera (JB):                0.644
Skew:                           0.017 

In [43]:
modelo = 'np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + qt_pessoas_residencia + qtd_filhos + tempo_emprego'
md = smf.ols(modelo, data = pr)
reg6 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 1
                        , alpha =  0.05)
reg6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     2142.
Date:                Fri, 06 Jan 2023   Prob (F-statistic):               0.00
Time:                        11:20:10   Log-Likelihood:                -13726.
No. Observations:               12427   AIC:                         2.746e+04
Df Residuals:                   12424   BIC:                         2.749e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               7.4981      0.012    643.175      0.000       7.475       7.521
C(sexo)[T.M]                            0.7772      0.014     56.582      0.000       0.750       0.804
C(posse_de_veiculo)[T.True]                  0          0        nan        nan           0           0
C(posse_de_imovel)[T.True]                   0          0        nan        nan           0           0
C(tipo_renda)[T.Bolsista]                    0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                  0          0        nan        nan           0           0
C(tipo_renda)[T.Pensionista]                 0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]            0          0        nan        nan           0           0
C(educacao)[T.Pós graduação]                 0          0        nan        nan           0           0
C(educacao)[T.Secundário]                    0          0        nan        nan           0           0
C(educacao)[T.Superior completo]             0          0        nan        nan           0           0
C(educacao)[T.Superior incompleto]           0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                  0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                  0          0        nan        nan           0           0
C(estado_civil)[T.União]                     0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                     0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                   0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]          0          0        nan        nan           0           0
qt_pessoas_residencia                        0          0        nan        nan           0           0
qtd_filhos                                   0          0        nan        nan           0           0
tempo_emprego                           0.0628      0.001     63.805      0.000       0.061       0.065
==============================================================================
Omnibus:                        0.528   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.768   Jarque-Bera (JB):                0.513
Skew:                           0.015 

In [44]:
modelo = 'np.log(renda) ~ C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + qt_pessoas_residencia + qtd_filhos + tempo_emprego'
md = smf.ols(modelo, data = pr)
reg6 = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 1
                        , alpha =  0.1)
reg6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     1607.
Date:                Fri, 06 Jan 2023   Prob (F-statistic):               0.00
Time:                        11:20:23   Log-Likelihood:                -13725.
No. Observations:               12427   AIC:                         2.746e+04
Df Residuals:                   12423   BIC:                         2.750e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               7.4788      0.020    377.695      0.000       7.440       7.518
C(sexo)[T.M]                            0.7753      0.014     56.099      0.000       0.748       0.802
C(posse_de_veiculo)[T.True]                  0          0        nan        nan           0           0
C(posse_de_imovel)[T.True]                   0          0        nan        nan           0           0
C(tipo_renda)[T.Bolsista]                    0          0        nan        nan           0           0
C(tipo_renda)[T.Empresário]                  0          0        nan        nan           0           0
C(tipo_renda)[T.Pensionista]                 0          0        nan        nan           0           0
C(tipo_renda)[T.Servidor público]            0          0        nan        nan           0           0
C(educacao)[T.Pós graduação]                 0          0        nan        nan           0           0
C(educacao)[T.Secundário]                    0          0        nan        nan           0           0
C(educacao)[T.Superior completo]             0          0        nan        nan           0           0
C(educacao)[T.Superior incompleto]           0          0        nan        nan           0           0
C(estado_civil)[T.Separado]                  0          0        nan        nan           0           0
C(estado_civil)[T.Solteiro]                  0          0        nan        nan           0           0
C(estado_civil)[T.União]                     0          0        nan        nan           0           0
C(estado_civil)[T.Viúvo]                     0          0        nan        nan           0           0
C(tipo_residencia)[T.Casa]                   0          0        nan        nan           0           0
C(tipo_residencia)[T.Com os pais]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Comunitário]            0          0        nan        nan           0           0
C(tipo_residencia)[T.Estúdio]                0          0        nan        nan           0           0
C(tipo_residencia)[T.Governamental]          0          0        nan        nan           0           0
qt_pessoas_residencia                   0.0085      0.007      1.208      0.227      -0.005       0.022
qtd_filhos                                   0          0        nan        nan           0           0
tempo_emprego                           0.0629      0.001     63.786      0.000       0.061       0.065
==============================================================================
Omnibus:                        0.513   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.774   Jarque-Bera (JB):                0.497
Skew:                           0.015 

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
data = pdr
X = pd.DataFrame(pdr)
y = data.renda

In [26]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(X, y).fit()
       
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() 
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data). The types seen wereNone and const       float64
data_ref     object
dtype: object. The data was
0         8060.34
1         1852.15
2         2253.89
3         6600.77
4         6475.97
           ...   
14995     7990.58
14996    10093.45
14997      604.82
14998     3352.27
14999     1160.99
Name: renda, Length: 15000, dtype: float64
and
        const    data_ref
0        1.0  2015-01-01
1        1.0  2015-01-01
2        1.0  2015-01-01
3        1.0  2015-01-01
4        1.0  2015-01-01
...      ...         ...
14995    1.0  2016-03-01
14996    1.0  2016-03-01
14997    1.0  2016-03-01
14998    1.0  2016-03-01
14999    1.0  2016-03-01

[15000 rows x 2 columns]
before. After,
[8060.34 1852.15 2253.89 ...  604.82 3352.27 1160.99]
[[1.0 '2015-01-01']
 [1.0 '2015-01-01']
 [1.0 '2015-01-01']
 ...
 [1.0 '2016-03-01']
 [1.0 '2016-03-01']
 [1.0 '2016-03-01']].